In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt



tickers = {
    "SPC": "^GSPC",  # Cap-weighted index
    "SPE": "RSP"     # Equal-weight ETF
}
start_date = "2005-01-01"
end_date = "2025-05-31"


In [2]:
### yahoo finance doesn't work:
spc = pd.read_csv("^GSPC_mon.csv", index_col=0, parse_dates=True)
spe = pd.read_csv("RSP_mon.csv", index_col=0, parse_dates=True)

# Combine into a single DataFrame
close_df = pd.DataFrame({
    "SPC": spc["Close"],
    "SPE": spe["Close"]
}).dropna()

# Compute returns and innovation portfolio
returns_df = close_df.pct_change().dropna()
returns_df["Innovation Portfolio"] = returns_df["SPC"] - returns_df["SPE"]
returns_df = returns_df.iloc[:-1]

In [3]:
#upload ff 3 factor data

ff_df = pd.read_csv("F-F_Research_Data_5_Factors_2x3.csv", index_col=0, skiprows=3)
ff_df = ff_df[ff_df.index.astype(str).str.match(r"^\d{6}$")]
ff_df.index = pd.to_datetime(ff_df.index.astype(str), format="%Y%m")
ff_df = ff_df.reindex(returns_df.index).dropna()

ff_df = ff_df.apply(pd.to_numeric, errors='coerce') / 100

merged_df = returns_df.join(ff_df, how='left')


excess_rets_df = pd.DataFrame()
excess_rets_df["SPC"] = merged_df["SPC"] - merged_df["RF"]
excess_rets_df["SPE"] = merged_df["SPE"] - merged_df["RF"]
excess_rets_df["Innovation Portfolio"] = merged_df["Innovation Portfolio"] - merged_df["RF"]
excess_rets_df["Mkt-RF"] = merged_df["Mkt-RF"]
excess_rets_df["SMB"] = merged_df["SMB"]
excess_rets_df["HML"] = merged_df["HML"]
excess_rets_df["RMW"] = merged_df["RMW"]
excess_rets_df["CMA"] = merged_df["CMA"]


In [4]:
#ffm = pd.read_csv(r"C:\Users\Username\OneDrive\Desktop\Invesco_project\F-F_Momentum_Factor.csv",skiprows= 13, nrows = 1180)
ffm = pd.read_csv("F-F_Momentum_Factor.csv",skiprows= 13, nrows = 1180)
ffm.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

ffm["Date"] = pd.to_datetime(ffm["Date"], format="%Y%m")

ffm.set_index("Date", inplace=True)

ffm = ffm.astype(float) / 100

In [5]:
ff_df = pd.merge(ff_df, ffm, on="Date", how="inner")

In [6]:
ff_df

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
2005-07-01,0.0392,0.0276,-0.0078,-0.0119,-0.0078,0.0024,0.0003
2005-08-01,-0.0120,-0.0094,0.0134,-0.0196,0.0038,0.0030,0.0207
2005-09-01,0.0048,-0.0031,0.0078,0.0026,-0.0061,0.0029,0.0346
2005-10-01,-0.0204,-0.0150,0.0043,-0.0070,-0.0126,0.0027,-0.0119
2005-11-01,0.0361,0.0090,-0.0118,-0.0079,-0.0109,0.0031,0.0034
...,...,...,...,...,...,...,...
2024-12-01,-0.0315,-0.0384,-0.0300,0.0191,-0.0121,0.0037,-0.0030
2025-01-01,0.0280,-0.0122,0.0162,-0.0233,-0.0324,0.0037,0.0095
2025-02-01,-0.0243,-0.0493,0.0491,0.0109,0.0306,0.0033,-0.0082


In [7]:

excess_rets_df["Mom"] = ff_df["Mom"]
excess_rets_df

,SPC,SPE,Innovation Portfolio,Mkt-RF,SMB,HML,RMW,CMA,Mom
Date,,,,,,,,,
2005-07-01,0.033568,0.051198,-0.020030,0.0392,0.0276,-0.0078,-0.0119,-0.0078,0.0003
2005-08-01,-0.014222,-0.011729,-0.005493,-0.0120,-0.0094,0.0134,-0.0196,0.0038,0.0207
2005-09-01,0.004049,-0.000560,0.001709,0.0048,-0.0031,0.0078,0.0026,-0.0061,0.0346
2005-10-01,-0.020441,-0.026918,0.003777,-0.0204,-0.0150,0.0043,-0.0070,-0.0126,-0.0119
2005-11-01,0.032086,0.042415,-0.013429,0.0361,0.0090,-0.0118,-0.0079,-0.0109,0.0034
...,...,...,...,...,...,...,...,...,...
2024-12-01,-0.028690,-0.069738,0.037348,-0.0315,-0.0384,-0.0300,0.0191,-0.0121,-0.0030
2025-01-01,0.023316,0.034241,-0.014624,0.0280,-0.0122,0.0162,-0.0233,-0.0324,0.0095
2025-02-01,-0.017542,-0.009149,-0.011693,-0.0243,-0.0493,0.0491,0.0109,0.0306,-0.0082


In [16]:
def compute_rolling_betas_and_alpha(
    excess_returns_df,
    portfolio_col,
    window=36,
    beta_MKT=False,
    beta_SMB=True,
    beta_HML=True,
    beta_RMW=True,
    beta_CMA=True,
    beta_Momentum=False
):
    factor_flags = {
        "Mkt-RF": beta_MKT,
        "SMB": beta_SMB,
        "HML": beta_HML,
        "RMW": beta_RMW,
        "CMA": beta_CMA,
        "Mom": beta_Momentum
    }

    selected_factors = [factor for factor, include in factor_flags.items() if include]

    results = {
        "alpha": [],
        "r_squared": [],
        "alpha_tstat": [],
        "alpha_pval": [],
        "sortino": []
    }

    for factor in selected_factors:
        results[f"beta_{factor}"] = []

    index = []

    for i in range(window, len(excess_returns_df)):
        window_df = excess_returns_df.iloc[i - window:i]

        y = window_df[portfolio_col]
        X = window_df[selected_factors]
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit()

        results["alpha"].append(model.params.get("const", float("nan")))
        results["r_squared"].append(model.rsquared)
        results["alpha_tstat"].append(model.tvalues.get("const", float("nan")))
        results["alpha_pval"].append(model.pvalues.get("const", float("nan")))

        downside_std = y[y < 0].std()
        sortino = y.mean() / downside_std if downside_std != 0 else float("nan")
        results["sortino"].append(sortino)

        for factor in selected_factors:
            results[f"beta_{factor}"].append(model.params.get(factor, float("nan")))

        index.append(excess_returns_df.index[i])

    return pd.DataFrame(results, index=index)


In [17]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

In [21]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import HTML

def plot_rolling(window=12, 
                 factors=["Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom"],
                 table_metrics=["r_squared", "alpha_tstat", "alpha_pval", "sortino"]):

    selected_df = compute_rolling_betas_and_alpha(
        excess_rets_df,
        portfolio_col="Innovation Portfolio",
        window=window,
        beta_MKT="Mkt-RF" in factors,
        beta_SMB="SMB" in factors,
        beta_HML="HML" in factors,
        beta_RMW="RMW" in factors,
        beta_CMA="CMA" in factors,
        beta_Momentum="Mom" in factors
    )

    # ---------- Plot alpha and betas ----------
    plot_cols = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
    selected_df[plot_cols].plot(figsize=(12, 6), title=f"{window}-Month Rolling Alpha & Betas")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ---------- Summary table ----------
    # Compute all available metrics
    all_metrics = {
        "Mean": selected_df.mean(),
        "Std Dev": selected_df.std(),
        "r_squared": selected_df["r_squared"].mean() if "r_squared" in selected_df.columns else None,
        "alpha_tstat": selected_df["alpha_tstat"].mean() if "alpha_tstat" in selected_df.columns else None,
        "alpha_pval": selected_df["alpha_pval"].mean() if "alpha_pval" in selected_df.columns else None,
        "sortino": selected_df["sortino"].mean() if "sortino" in selected_df.columns else None
    }

    # Filter metrics based on user selection
    selected_metrics = {k: v for k, v in all_metrics.items() if k in table_metrics and v is not None}

    # Build summary DataFrame for selected rows and columns
    row_names = ["alpha"] + [f"beta_{f}" for f in factors if f"beta_{f}" in selected_df.columns]
    summary = pd.DataFrame({metric: selected_df[row_names].mean() if metric == "Mean"
                            else selected_df[row_names].std() if metric == "Std Dev"
                            else [selected_metrics[metric]] * len(row_names)
                            for metric in selected_metrics})

    summary.index = row_names
    summary = summary.round(4)

    # Store globally
    global latest_summary_table
    latest_summary_table = summary

    display(HTML("<h4>Average Results from Rolling Factor Mode</h4>"))
    display(summary)


interact(
    plot_rolling,
    window=widgets.IntSlider(min=6, max=60, step=6, value=12),
    factors=widgets.SelectMultiple(
        options=["Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom"],
        value=("SMB", "HML", "RMW", "CMA", "Mom"),
        description='Factors',
        style={'description_width': 'initial'}
    ),
    table_metrics=widgets.SelectMultiple(
        options=["Mean", "Std Dev", "r_squared", "alpha_tstat", "alpha_pval", "sortino"],
        value=("Mean", "Std Dev"),
        description='Summary Columns',
        style={'description_width': 'initial'}
    )

)



interactive(children=(IntSlider(value=12, description='window', max=60, min=6, step=6), SelectMultiple(descrip…

<function __main__.plot_rolling(window=12, factors=['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom'], table_metrics=['r_squared', 'alpha_tstat', 'alpha_pval', 'sortino'])>